<a href="https://colab.research.google.com/github/lsgotti/TCC_Python/blob/main/Oversample_offline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#carrega imports e instala o scikit-multiflow
!pip install -U scikit-multiflow
# Imports
from skmultiflow.data import FileStream
from skmultiflow.data import SEAGenerator
from skmultiflow.evaluation import EvaluatePrequential
from skmultiflow.bayes import NaiveBayes
from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.trees import HoeffdingAdaptiveTreeClassifier
from skmultiflow.trees import ExtremelyFastDecisionTreeClassifier
from skmultiflow.meta import LearnPPNSEClassifier
from skmultiflow.drift_detection import ADWIN
from skmultiflow.meta import OzaBaggingClassifier

from sklearn.linear_model import SGDClassifier

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

     |████████████████████████████████| 1.1 MB 35.0 MB/s 


In [ ]:
#carrega imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
import pandas as pd
from sklearn import datasets, linear_model
from matplotlib import pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

from google.colab import files

import numpy as np
from skmultiflow.data.file_stream import FileStream
from skmultiflow.data.data_stream import DataStream
from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.evaluation import EvaluatePrequential
import pandas as pd
import io
from google.colab import files
from skmultiflow.trees import HoeffdingTree
from datetime import datetime, date

In [ ]:
#segunda parte
#importa arquivo para teste
uploaded = files.upload()
#Importa o arquivo para o colab
import io


Saving tomcat2.csv to tomcat2.csv


In [ ]:
#abre o arquivo no dataframe
df = pd.read_csv(io.BytesIO(uploaded['tomcat2.csv']))

In [ ]:
#abre arquivo na stream stream
stream = FileStream("tomcat2.csv")

In [ ]:
np.set_printoptions(formatter={'float_kind':'{:25f}'.format})

#Etapa de upsampling

from sklearn.utils import resample

#Divide o dataframe em codigos 0 e 1, apenas o 1 sera usado

m=df.code==0
df_code0=df[m]
df_code1=df[~m]

print(df_code1.contains_bug.value_counts())

# Separa classes minoritarias e majoritarias
df_majority = df_code1[df_code1.contains_bug==0]
df_minority = df_code1[df_code1.contains_bug==1]


# Upsample de classe minoritaria
df_minority_upsampled = resample(df_minority, replace=True, n_samples=len(df_majority), random_state=123)


# Combina classe majoritaria e minoritaria
df_upsampled_partial = pd.concat([df_majority, df_minority_upsampled])

print(df_upsampled_partial.contains_bug.value_counts())
df_upsampled = pd.concat([df_upsampled_partial, df_code0])
print(df_upsampled.contains_bug.value_counts())
df_upsampled

NameError: ignored

In [ ]:
#Embaralha dataframe
df_upsampled = df_upsampled.sample(frac=1)


In [ ]:
#Abre stream com dataframe upsampled
stream = DataStream(df_upsampled)

In [ ]:
#Inicializa variaveis e lista que serão usadas, deve ser rodado a cada teste

indexStream=1

periodicidadeTreinamento = 100

w = 100

PREDICT = 0

TRAIN = 1

#classifier = HoeffdingTree()
classifier = OzaBaggingClassifier(base_estimator=HoeffdingTree(), n_estimators=20)
#classifier = LearnPPNSEClassifier()
#classifier = HoeffdingAdaptiveTreeClassifier()
#classifier = ExtremelyFastDecisionTreeClassifier()
correctness_dist = []
i = 0

poolCleanInstances = []
poolDefectInducingInstances = []
poolTrainInstances = []
poolTrainClasses = []
poolTrainInstances2= []


/usr/local/lib/python3.7/dist-packages/skmultiflow/trees/hoeffding_tree.py:35: FutureWarning: 'HoeffdingTree' has been renamed to 'HoeffdingTreeClassifier' in v0.5.0.
The old name will be removed in v0.7.0
  "The old name will be removed in v0.7.0", category=FutureWarning)


In [ ]:
#Cria a classe de recall
class Recall:
  recall0 = 0
  recall1 = 0

  b0 = 0
  b1 = 0

  alpha = 0.999

  def updateRec0(self, value):
    self.recall0 = value + self.alpha * self.recall0
    self.b0 =  self.alpha * self.b0 + 1.0;

  def getRecall0(self):
    if self.b0 > 0:
      #print("b0>0")
      #print(self.b0)
      return self.recall0/self.b0
    else:
      #print("b0 not 0")
      #print(self.b0)
      return 0

  def updateRec1(self, value):
    self.recall1 = value +  self.alpha * self.recall1
    self.b1 =  self.alpha * self.b1 + 1.0;

  def getRecall1(self):
    if self.b1 > 0:
      #print("b1>0")
      #print(self.b1)
      return self.recall1/self.b1
    else:
      #print("b1 not 0")
      #print(self.b1)
      return 0

In [ ]:
#Cria as listas de recall
import math

recall = Recall()

recall0list = [0]
recall1list = [0]


In [ ]:
TP = 0
TN = 0
FP = 0
FN = 0

ct = 0
isInicio = True
#Roda enquanto ainda tem instancias na stream
while (stream.has_more_samples()):
  ct=ct+1
  #Incrementa o indexStream para comparar com o w e treinar classificador
  indexStream = indexStream +1
  #X são as caracteristicas da instancia e control é o code (0 ou 1)
  X, control = stream.next_sample()
  if(isInicio):
    #print("Nao faz nada nas primeiras w instancias")
    if(indexStream % periodicidadeTreinamento == 0):
      for n in range(len(poolTrainInstances)):
        poolTrainInstances[n] = poolTrainInstances[n][0]

      poolTrainClasses = [row[15] for row in poolTrainInstances]

      for n in range(len(poolTrainInstances)):
        poolTrainInstances[n] = poolTrainInstances[n][1:14]

      clf = LogisticRegression(solver='liblinear',multi_class='ovr').fit(poolTrainInstances, poolTrainClasses)
      indexStream=1
      isInicio = False
    else:
      #Incrementa o indexStream para comparar com o w e treinar classificador
      #X são as caracteristicas da instancia e control é o code (0 ou 1)
      current_timestamp = X[0,0]
      original_timestamp = X[0,16]
      current_date = datetime.fromtimestamp(current_timestamp)
      original_date = datetime.fromtimestamp(original_timestamp)
      classe = int(X[0,15])
      X2 = [X[0 ,1:14]]

      #Se control for predict
      if(control == PREDICT):
        correctness_dist.append(0)
        #Entra se a classe eh 0 e da update no recall 0
        if (classe == 0):
          FN = FN + 1
          recall.updateRec0(0)
          recall0list.append(recall.getRecall0())
          #print("FN ERROU")
        #Entra se a classe eh 1 e da update no recall 1
        else:
          FP = FP + 1
          recall.updateRec1(0)
          recall1list.append(recall.getRecall1())
          #print("FN ERROU")

      #se control for train
      if(control == TRAIN):
        #print("train")
        #adiciona a instancia na lista de instancias a serem treinadas, essas instancias so serao treinadas quando indexStream chegar no valor de periodicidadeTreinamento
        poolTrainInstances.append(X)
        if(X[0,2] == 0):
              # update the tree
          poolCleanInstances.append(X)
        if(X[0,2] == 1):
          poolDefectInducingInstances.append(X)
  else:
    #Incrementa o indexStream para comparar com o w e treinar classificador
    indexStream = indexStream +1
    #X são as caracteristicas da instancia e control é o code (0 ou 1)
    X, control = stream.next_sample()
    print(len(X), control, ct)
    current_timestamp = X[0,0]
    original_timestamp = X[0,16]
    current_date = datetime.fromtimestamp(current_timestamp)
    original_date = datetime.fromtimestamp(original_timestamp)
    classe = int(X[0,15])
    X2 = [X[0 ,1:14]]

    #Se control for predict
    if(control == PREDICT):
      #print("predict")
      #usa o metodo predict e guarda em prediction
      prediction = clf.predict(X2)
      #caso prediction acerte, faz update nas listas e no recall
      if classe == prediction[0]:
        correctness_dist.append(1)
        #Entra se a classe eh 0 e da update no recall 0
        if (classe == 0):
          if not recall1list:
            recall1list.append(0)
          if recall1list:
            recall1list.append(recall1list[-1])
          TN = TN + 1
          #print("TN ACERTOU")
          recall.updateRec0(1)
          recall0list.append(recall.getRecall0())
          #caso prediction erre, faz update nas listas e no recall
        #Entra se a classe eh 1 e da update no recall 1
        else:
          if not recall1list:
            recall1list.append(0)
          if recall1list:
            recall1list.append(recall1list[-1])
          TP = TP + 1
          recall.updateRec1(1)
          recall1list.append(recall.getRecall1())

          #print("TP ACERTOU")
      #caso prediction acerte, faz update nas listas e no recall
      else:
        correctness_dist.append(0)
        #Entra se a classe eh 0 e da update no recall 0
        if (classe == 0):
          FN = FN + 1
          recall.updateRec0(0)
          recall0list.append(recall.getRecall0())
          #print("FN ERROU")
        #Entra se a classe eh 1 e da update no recall 1
        else:
          FP = FP + 1
          recall.updateRec1(0)
        recall1list.append(recall.getRecall1())
          #print("FN ERROU")

      #poolCleanInstances.append(X)
    #se control for train
    if(control == TRAIN):
      #print("train")
      #adiciona a instancia na lista de instancias a serem treinadas, essas instancias so serao treinadas quando indexStream chegar no valor de periodicidadeTreinamento
      poolTrainInstances.append(X)
      if(X[0,2] == 0):
            # update the tree
        poolCleanInstances.append(X)
      if(X[0,2] == 1):
        poolDefectInducingInstances.append(X)

    #Caso indexStream chegue no valor de periodicidadeTreinamento, realizamos o treinamento percorrendo todas as instancias na lista poolTrainInstances e treinando o classificador com partial_fit
    if(indexStream % periodicidadeTreinamento == 0):
      #resetar  o classificador
      #nao resetar o pool de instancias
      #treinar o classificador com ten fold cross validation
      #fazer isso pra 100, 500 e 1000 periodicidade
      poolTrainClasses = [row[15] for row in poolTrainInstances]
      clf = LogisticRegression(solver='liblinear',multi_class='ovr').fit(poolTrainInstances, poolTrainClasses)
      print("reseta clf")
      indexStream=1
    #poolTrainInstances.clear()

print("TP: "+str(TP))
print("FP: "+str(FP))
print("TN: "+str(TN))
print("FN: "+str(FN))
print(len(correctness_dist))
print(sum(correctness_dist))

ValueError: ignored

In [ ]:
#Organiza os recalls em uma lista e calcula gmean e por ultimo baixa como csv
for i in range(abs(len(recall0list) - len(recall1list))):
  if len(recall0list) > len(recall1list):
    recall1list.append(recall1list[-1])
  else:
    recall0list.append(recall0list[-1])

print(len(recall1list))
print(len(recall0list))
print(len(recall1list)+len(recall0list))

gmeanList = []
for i in range(len(recall0list)):
  gmeanList.append(math.sqrt(recall0list[i] * recall1list[i]))

print(len(gmeanList))

from google.colab import files

results = pd.DataFrame(
    {'recall0': recall0list,
     'recall1': recall1list,
     'gmean': gmeanList
    })

results.to_csv('results.csv')
files.download('results.csv')